Thymio mini project

In [ ]:
!pip install --upgrade tdmclient

In [ ]:
# import libraries
import time
from tdmclient import ClientAsync
import numpy as np
import math

# import project files
import constants as cst
import vision
import global_navigation as gn
#import local_navigation as ln
import extended_Kalman_filter as eKf
import geometry as geo

#connect to the robot
client = ClientAsync()
node = await client.wait_for_node()
await node.lock()

In [ ]:
await node.set_variables({
            'leds.bottom.left': [0, 0, 0],
            'leds.bottom.right': [0, 0, 0],
            'leds.circle': [0, 0, 0, 0, 0, 0, 0, 0],
            'leds.top': [0, 0, 0],
        })

In [ ]:
# Set the motor speeds
async def stop_robot():
    """
    Stop the robot. Set the motor speed of each wheel to zero.
    """
    await node.set_variables({
            "motor.right.target": [0],
            "motor.left.target": [0],
        })

async def move_robot(right_speed, left_speed):
    """
    Turn the robot wheels to the given speeds.
    Parameter: - right_speed   : right wheel speed in [thymio speed]
               - left_speed    : left wheel speed in [thymio speed]
    """
    await node.set_variables({
            "motor.right.target": [right_speed],
            "motor.left.target": [left_speed],
        })

In [ ]:
# Get the proximity sensor values
async def get_prox_sensors():
    """
    Get the sensor measurements of a the proximity sensors.
    Returns: - prox_sensors    : numpy array of fron proximity sensors measurements
             - detected        : boolean, true if any sensors measured a higher value than threshold
    """
    
    await node.wait_for_variables({str('prox.horizontal')})
    prox_sensors = np.asarray(list(node['prox.horizontal']))
    prox_sensors = prox_sensors[0:5]

    prox_sensors_bool = (prox_sensors >= np.ones(5) * 20) #cst.THRESHOLD_PROX_SENSOR
    detected = False
    if(any(prox_sensors_bool)):
        detected = True

    return prox_sensors, detected

# Get the motor speed measurement values
async def get_motor_speed_meas():
    """
    Get the motor speed measurements
    Returns: - speed_left_meas    : left wheel speed in [thymio speed]
             - speed_right_meas   : right wheel speed in [thymio speed]
    """
    
    await node.wait_for_variables({str('motor.right.speed')})
    speed_right_meas = node['motor.right.speed']
    await node.wait_for_variables({str('motor.left.speed')})
    speed_left_meas = node['motor.left.speed']

    return speed_right_meas, speed_left_meas

In [ ]:
# test thymio
# move the robot forward and get prox values.
await move_robot(50, 50)
dt = 0
old_time = time.time()
while(dt < 2):
    y,d = await get_prox_sensors()
    print(y)
    print(d)
    await client.sleep(0.1)
    dt = time.time() - old_time
await stop_robot()

In [ ]:
#VISION AND GLOBAL NAVIGATION -TEST SANS VISION
##obstacles, goal_centroid = vision.vision_obstacles_and_goal()
##robot_centroid, robot_direction = vision.vision_robot()

intermediate_goals = [(20,20), (2000, 2000)]
robot_position = [0.0,0.0]
robot_orientation = math.pi / 2.0
robot_speed = [0.0,0.0]
speed_correction = 0

robot_centroid = [0.0,0.0]
robot_direction = (0.0,1.0)
shortest_path = intermediate_goals
obstacles = [] 


merge_need = True
visualize = True
#shortest_path, obstacles = gn.global_navigation(tuple(robot_centroid), obstacles, tuple(goal_centroid), merge_need, visualize)
#print("glob nav path :", shortest_path)


def pixels_to_cm(coordinates, pixel_to_cm_ratio=0.05406):
    # Convert each coordinate in the list or individual point
    if isinstance(coordinates[0], (list, tuple)):
        # List of coordinates
        converted_coordinates = [
            (int(x * pixel_to_cm_ratio), int(y * pixel_to_cm_ratio)) for x, y in coordinates
        ]
    else:
        # Single coordinate
        converted_coordinates = (int(coordinates[0] * pixel_to_cm_ratio), int(coordinates[1] * pixel_to_cm_ratio))
    
    return converted_coordinates



shortest_path = pixels_to_cm(shortest_path)


robot_centroid = pixels_to_cm(robot_centroid)
print(robot_centroid)
print(shortest_path)

#obstacles = np.array(obstacles)

In [ ]:
#ROBOT CONTROL AND LOCALISATION

# state variables
global_state = 'rotation'

#speed_correction = 12.5 #calibration cst for rotation counterclockwise
speed_correction = 14 #calibration cst for rotation counterclockwise
#speed_correction = 0.75 #0.65 #calibration cst for translation

intermediate_goals = shortest_path
intermediate_goals.pop(0)
robot = [(robot_centroid[0],robot_centroid[1])]
robot_orientation = np.arctan2(robot_direction[1], robot_direction[0])

intermediate_goals = [(0, 10), (-10, 10), (-10, 0), (0,0)]
intermediate_goals = [(56, 34), (66, 34), (87, 26)]
robot = [(34.0,28.0)]
robot_orientation = 1.6
robot_speed = [100,100]

Kfilter = eKf.Kalman(robot[0][0], robot[0][1], robot_orientation)

#robot_speed = [30, -30] #robot speed for rotation (counterclockwise)
#robot_speed = [-30, 30] #robot speed for rotation (clockwise)
#robot_speed = [50, 50] #robot speed for translation
#await move_robot(robot_speed[0], robot_speed[1])

old_time = 0.0
printkal = False
iter = 0
while(True):
    iter = iter + 1
    print(iter, global_state)

    if intermediate_goals == []:
        await stop_robot()
        print("stop robot")
        break

    # get robot sensor values
    prox_sensors, detected = await get_prox_sensors()
    # test if local (with proximity)
    if(detected):
        global_state = 'local_avoidance'

    
    speed_right_meas, speed_left_meas = await get_motor_speed_meas()
    print(speed_right_meas, speed_left_meas, "  ", robot_speed[0], robot_speed[1])
    speed_right_meas = speed_right_meas * speed_correction
    speed_left_meas = speed_left_meas * speed_correction
    
    #speed_right_meas = robot_speed[0] * speed_correction
    #speed_left_meas = robot_speed[1] * speed_correction

    Kfilter.measurement_wheels(speed_right_meas / 18.1818, speed_left_meas / 18.1818)

    if iter == 0:                   # first time
        old_time = time.time()
    dt = time.time() - old_time     # get time difference for Kalman
    old_time = time.time()

    Kfilter.dt_update(dt)
    Kfilter.Kalman_filter()
    #print(dt, time.time(), old_time)
    if printkal:
        print("Kalman: ", Kfilter.Mu[0][0], Kfilter.Mu[1][0], Kfilter.Mu[2][0], Kfilter.Mu[3][0], Kfilter.Mu[4][0])
        printkal = False
    #print("Kalman Sigma: ", Kfilter.Sigma)
    #print("Kalman y: ", Kfilter.y)
    
    robot.append((Kfilter.Mu[0][0], Kfilter.Mu[1][0]))
    
    if global_state == 'rotation':
        angle = np.arctan2(intermediate_goals[0][1] - Kfilter.Mu[1], intermediate_goals[0][0] - Kfilter.Mu[0]) - Kfilter.Mu[2]
        angle = geo.center_angle(angle)
        print(Kfilter.Mu[2], np.arctan2(intermediate_goals[0][1] - Kfilter.Mu[1], intermediate_goals[0][0] - Kfilter.Mu[0]), angle)
        #if(abs(Kfilter.Mu[2][0]) > np.pi / 4.0):        #rotation condition
        if(abs(angle) < 0.08):
            await stop_robot()
            global_state = 'translation'
            continue

        if(angle > 0):
            robot_speed = [30, -30]         # counterclockwise
            speed_correction = 2 #32 #14
        else:
            robot_speed = [-30, 30]         # clockwise
            speed_correction = 2 #12.5

    elif global_state == 'translation':
        robot_speed = [50, 50]
        speed_correction = 0.75
        print("Error:", np.sqrt((intermediate_goals[0][0] - Kfilter.Mu[0])**2 + (intermediate_goals[0][1] - Kfilter.Mu[1])**2))
        #if(np.sqrt(Kfilter.Mu[0][0]**2 + Kfilter.Mu[1][0]**2) > 10.0):     #translation condition
        
        if np.sqrt((intermediate_goals[0][0] - Kfilter.Mu[0])**2 + (intermediate_goals[0][1] - Kfilter.Mu[1])**2) <= 6:
            print("je m'arrete")
            await stop_robot()
            intermediate_goals.pop(0)
            global_state = 'camera'
            continue
        
        
        angle = np.arctan2(intermediate_goals[0][1] - Kfilter.Mu[1], intermediate_goals[0][0] - Kfilter.Mu[0]) - Kfilter.Mu[2]
        angle = geo.center_angle(angle)
        if abs(angle) > math.pi / 4.0:
            await stop_robot()
            global_state = 'rotation'
            continue

    elif global_state == 'local_avoidance':
        # call local_avoidance fct (= add obstacle on global map and recompute global nav)
        robot_direction = (math.cos(Kfilter.Mu[2][0]), math.sin(Kfilter.Mu[2][0]))
        # Object = ln.local_nav([Kfilter.Mu[0][0], Kfilter.Mu[1][0]], robot_direction, prox_sensors)
        object_robot_coords = ln.object_robot_coordinates(prox_sensors)
        print("obstacle in robot coord", object_robot_coords)
        print([Kfilter.Mu[0][0], Kfilter.Mu[1][0]])
        object_ground_coords = ln.robot_to_ground_coords([Kfilter.Mu[0][0], Kfilter.Mu[1][0]], robot_direction, object_robot_coords)
        print("obstacle in ground coord", object_ground_coords)
        Object = ln.safety_zone(object_ground_coords)
        print("obstacle square", Object)

        obstacles = obstacles.append(Object)
        print("la liste des obstacles",obstacles)

        await stop_robot()
        break
        
    global_state = 'rotation'
    time.sleep(0.1)


    


    

In [ ]:
await stop_robot()

In [ ]:
import matplotlib.pyplot as plt

# Unpack the tuples into separate lists for x and y coordinates
x_coords, y_coords = zip(*robot)

# Plot the points
plt.scatter(x_coords, y_coords)
plt.scatter(robot[0][0], robot[0][1], color = "green")
plt.scatter(robot[-1][0], robot[-1][1], color = "red")

# Add labels and title
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.title('2D Points Plot')

# Display the plot
plt.show()
print(robot)